In [4]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.callbacks import EarlyStopping

from utils.dataloader import load_data_npy, PreprocessedDataGenerator
from utils.model_utils import save_model_and_history, save_test_results

In [2]:
images_train, categories_train, images_val, categories_val, images_test, categories_test = load_data_npy()

# --- Generadores ---
def preprocess_fn(x):
    return vgg_preprocess(x)

# Generadores adaptados
train_gen = PreprocessedDataGenerator(images_train, categories_train, preprocess_fn=preprocess_fn, resize_to=(224, 224), batch_size=16)
val_gen = PreprocessedDataGenerator(images_val, categories_val, shuffle=False, preprocess_fn=preprocess_fn, resize_to=(224, 224), batch_size=16)
test_gen = PreprocessedDataGenerator(images_test, categories_test, shuffle=False, preprocess_fn=preprocess_fn, resize_to=(224, 224), batch_size=16)

print(f"Train: {images_train.shape}, Validation: {images_val.shape}, Test: {images_test.shape}")

Train: (10220, 150, 150, 3), Validation: (2555, 150, 150, 3), Test: (4259, 150, 150, 3)


In [ ]:
# Opción 2: VGG16 (más pesado, buena extracción de características)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar capas convolucionales del modelo base
base_model.trainable = False  # para feature extraction

# --- Construir modelo con cabeza personalizada ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- Entrenamiento ---
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=30,
                    callbacks=[early_stop])

# --- Evaluación ---
test_loss, test_acc = model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_acc:.4f}   |  Test Loss: {test_loss:.4f}")

# --- Guardado ---
save_model_and_history(model, history, model_path='transfer_model_vgg16')
save_test_results('transfer_model_vgg16', test_loss, test_acc)

Epoch 1/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2497s 4s/step - accuracy: 0.1736 - loss: 1.9277 - val_accuracy: 0.2830 - val_loss: 1.7487
Epoch 2/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2415s 4s/step - accuracy: 0.2324 - loss: 1.7397 - val_accuracy: 0.3100 - val_loss: 1.6644
Epoch 3/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2549s 4s/step - accuracy: 0.2611 - loss: 1.6702 - val_accuracy: 0.2920 - val_loss: 1.6060
Epoch 4/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2481s 4s/step - accuracy: 0.2841 - loss: 1.6214 - val_accuracy: 0.3241 - val_loss: 1.5631
Epoch 5/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2492s 4s/step - accuracy: 0.2969 - loss: 1.5847 - val_accuracy: 0.3225 - val_loss: 1.5511
Epoch 6/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2442s 4s/step - accuracy: 0.3007 - loss: 1.5852 - val_accuracy: 0.3389 - val_loss: 1.5244
Epoch 7/30
639/639 ━━━━━━━━━━━━━━━━━━━━ 2481s 4s/step - accuracy: 0.3207 - loss: 1.5514 - val_accuracy: 0.3256 - val_loss: 1.5296
Epoch 8/30
294/639 ━━━━━━━━━━━━━━━━━━━━ 17:58 3s/step - accuracy: 0.3321 - loss: 1.5540